In [2]:
# NOTEBOOK TO RUN THE FINAL DIFFUSION MODEL WITH ATTENTION AT ALL LEVELS. YOU SHOULD RUN THIS ON A GPU OR ANYWHERE WITH CUDA 
# COMPATIBILITY AS GENERATION CAN BE VERY LONG ON A CPU. THE WEIGHTS ARE DOWNLOADED FROM MY GOOGLE DRIVE (Antoine). 

In [3]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install torch

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import gdown
import os

file_id = "1JKHdVVaw2ySOuuPJCcL94tnyHyavV4Xj"
out_path = "ema_weights_final_model.pth"

if not os.path.exists(out_path):
    gdown.download(
        f"https://drive.google.com/uc?id={file_id}",
        out_path,
        quiet=False,
    )
else:
    print("EMA weights already downloaded.")


EMA weights already downloaded.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision

device = "cuda" if torch.cuda.is_available() else "cpu"
print(torch.cuda.is_available()) # Make sure it prints true !

True


In [ ]:
import math
import matplotlib.pyplot as plt

# -------------------------
# Model (matches your checkpoint keys)
# -------------------------
class TimeEmbedding(nn.Module):
    """
    Sinusoidal timestep embedding -> small MLP.
    Produces a vector of size time_dim.
    """
    def __init__(self, dim=128, hidden_mult=4):
        super().__init__()
        self.dim = dim
        hidden = dim * hidden_mult
        self.mlp = nn.Sequential(
            nn.Linear(dim, hidden),
            nn.SiLU(),
            nn.Linear(hidden, dim),
        )

    def forward(self, t: torch.Tensor) -> torch.Tensor:
        # t: [B] float in [0, 1]
        half_dim = self.dim // 2
        freqs = torch.exp(
            torch.arange(half_dim, device=t.device, dtype=t.dtype)
            * -(torch.log(torch.tensor(10000.0, device=t.device, dtype=t.dtype)) / half_dim)
        )
        args = t[:, None] * freqs[None, :]  # [B, half_dim]
        emb = torch.cat([torch.sin(args), torch.cos(args)], dim=-1)  # [B, dim] if dim even
        if emb.shape[1] != self.dim:
            # if dim is odd, pad
            emb = F.pad(emb, (0, self.dim - emb.shape[1]))
        return self.mlp(emb)

class ResBlockFiLM(nn.Module):
    """
    ResBlock with FiLM conditioning from an embedding vector (time+label).
    Conditioning is applied as scale+shift after GroupNorm:
        h = GN(h) * (1 + scale) + shift
    """
    def __init__(self, in_ch, out_ch, emb_dim, groups=32, dropout=0.0):
        super().__init__()
        self.in_ch = in_ch
        self.out_ch = out_ch

        self.norm1 = nn.GroupNorm(groups, in_ch)
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)

        self.norm2 = nn.GroupNorm(groups, out_ch)
        self.dropout = nn.Dropout(dropout) if dropout > 0 else nn.Identity()
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)

        # project embedding -> (scale, shift)
        self.emb_proj = nn.Sequential(
            nn.SiLU(),
            nn.Linear(emb_dim, 2 * out_ch),
        )

        self.skip = nn.Identity() if in_ch == out_ch else nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x, emb):
        # x: [B,C,H,W], emb: [B, emb_dim]
        h = self.norm1(x)
        h = F.silu(h)
        h = self.conv1(h)

        h = self.norm2(h)
        scale_shift = self.emb_proj(emb)  # [B, 2*out_ch]
        scale, shift = torch.chunk(scale_shift, 2, dim=1)
        scale = scale[:, :, None, None]
        shift = shift[:, :, None, None]
        h = h * (1.0 + scale) + shift

        h = F.silu(h)
        h = self.dropout(h)
        h = self.conv2(h)

        return h + self.skip(x)

class SelfAttention2d(nn.Module):
    def __init__(self, channels, num_heads=4, gn_groups=32):
        super().__init__()
        assert channels % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = channels // num_heads
        self.scale = self.head_dim ** -0.5

        self.norm = nn.GroupNorm(gn_groups, channels)
        self.qkv = nn.Conv2d(channels, 3 * channels, kernel_size=1, bias=False)
        self.proj = nn.Conv2d(channels, channels, kernel_size=1)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.norm(x)
        qkv = self.qkv(h)  # [B,3C,H,W]
        q, k, v = qkv.chunk(3, dim=1)

        N = H * W
        q = q.view(B, self.num_heads, self.head_dim, N).permute(0, 1, 3, 2)  # [B,h,N,d]
        k = k.view(B, self.num_heads, self.head_dim, N)                      # [B,h,d,N]
        v = v.view(B, self.num_heads, self.head_dim, N).permute(0, 1, 3, 2)  # [B,h,N,d]

        attn = (q @ k) * self.scale
        attn = attn.softmax(dim=-1)

        out = attn @ v
        out = out.permute(0, 1, 3, 2).contiguous().view(B, C, H, W)
        out = self.proj(out)
        return x + out

class UNetCIFAR3Level_Attn_CFG(nn.Module):
    """
    ε-prediction U-Net with attention at 32x32, 16x16, 8x8 and 4x4,
    plus classifier-free guidance conditioning via null label = num_classes.
    """
    def __init__(
        self,
        time_dim=32,
        base_channels=128,
        img_channels=3,
        num_classes=10,
        attn_heads=4,
        dropout=0.0,
        gn_groups=32,
    ):
        super().__init__()
        self.time_dim = time_dim
        self.num_classes = num_classes
        self.null_label = num_classes

        self.time_mlp = TimeEmbedding(dim=time_dim)
        self.label_emb = nn.Embedding(num_classes + 1, time_dim)

        C = base_channels
        emb_dim = time_dim

        # Stem
        self.in_conv = nn.Conv2d(img_channels, C, 3, padding=1)

        # Encoder
        self.down1 = ResBlockFiLM(C, C, emb_dim, groups=gn_groups, dropout=dropout)
        self.attn32 = SelfAttention2d(C, num_heads=attn_heads, gn_groups=gn_groups)
        self.pool1 = nn.MaxPool2d(2)  # 32 -> 16

        self.down2 = ResBlockFiLM(C, 2 * C, emb_dim, groups=gn_groups, dropout=dropout)
        self.attn16 = SelfAttention2d(2 * C, num_heads=attn_heads, gn_groups=gn_groups)
        self.pool2 = nn.MaxPool2d(2)  # 16 -> 8

        self.down3 = ResBlockFiLM(2 * C, 4 * C, emb_dim, groups=gn_groups, dropout=dropout)
        self.attn8 = SelfAttention2d(4 * C, num_heads=attn_heads, gn_groups=gn_groups)
        self.pool3 = nn.MaxPool2d(2)  # 8 -> 4

        # Bottleneck
        self.bottleneck = ResBlockFiLM(4 * C, 8 * C, emb_dim, groups=gn_groups, dropout=dropout)
        self.attn4 = SelfAttention2d(8 * C, num_heads=attn_heads, gn_groups=gn_groups)

        # Decoder
        self.up3 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(8 * C, 4 * C, 3, padding=1),
        )
        self.dec3 = ResBlockFiLM(8 * C, 4 * C, emb_dim, groups=gn_groups, dropout=dropout)
        self.dec3_attn = SelfAttention2d(4 * C, num_heads=attn_heads, gn_groups=gn_groups)

        self.up2 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(4 * C, 2 * C, 3, padding=1),
        )
        self.dec2 = ResBlockFiLM(4 * C, 2 * C, emb_dim, groups=gn_groups, dropout=dropout)
        self.dec2_attn16 = SelfAttention2d(2 * C, num_heads=attn_heads, gn_groups=gn_groups)

        self.up1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode="nearest"),
            nn.Conv2d(2 * C, C, 3, padding=1),
        )
        self.dec1 = ResBlockFiLM(2 * C, C, emb_dim, groups=gn_groups, dropout=dropout)
        self.dec1_attn32 = SelfAttention2d(C, num_heads=attn_heads, gn_groups=gn_groups)

        self.out_norm = nn.GroupNorm(gn_groups, C)
        self.out_conv = nn.Conv2d(C, img_channels, 3, padding=1)

    def forward(self, x, t, y=None):
        B = x.shape[0]
        temb = self.time_mlp(t)

        if y is None:
            y = torch.full((B,), self.null_label, device=x.device, dtype=torch.long)
        yemb = self.label_emb(y)

        emb = temb + yemb

        h0 = self.in_conv(x)

        d1 = self.down1(h0, emb)
        d1 = self.attn32(d1)
        p1 = self.pool1(d1)

        d2 = self.down2(p1, emb)
        d2 = self.attn16(d2)
        p2 = self.pool2(d2)

        d3 = self.down3(p2, emb)
        d3 = self.attn8(d3)
        p3 = self.pool3(d3)

        b = self.bottleneck(p3, emb)
        b = self.attn4(b)

        u3 = self.up3(b)
        u3 = torch.cat([u3, d3], dim=1)
        u3 = self.dec3(u3, emb)
        u3 = self.dec3_attn(u3)

        u2 = self.up2(u3)
        u2 = torch.cat([u2, d2], dim=1)
        u2 = self.dec2(u2, emb)
        u2 = self.dec2_attn16(u2)

        u1 = self.up1(u2)
        u1 = torch.cat([u1, d1], dim=1)
        u1 = self.dec1(u1, emb)
        u1 = self.dec1_attn32(u1)

        out = self.out_norm(u1)
        out = F.silu(out)
        out = self.out_conv(out)
        return out


# -------------------------
# VPSDE + sampler
# -------------------------
class VPSDE:
    def __init__(self, beta_min=0.1, beta_max=20.0, T=1.0):
        self.beta_min = float(beta_min)
        self.beta_max = float(beta_max)
        self.T = float(T)

    def beta(self, t: torch.Tensor) -> torch.Tensor:
        return self.beta_min + t * (self.beta_max - self.beta_min)

    def int_beta(self, t: torch.Tensor) -> torch.Tensor:
        return self.beta_min * t + 0.5 * (self.beta_max - self.beta_min) * t**2

    def alpha(self, t: torch.Tensor) -> torch.Tensor:
        return torch.exp(-0.5 * self.int_beta(t))

    def sigma(self, t: torch.Tensor) -> torch.Tensor:
        a = self.alpha(t)
        return torch.sqrt((1.0 - a * a).clamp(min=1e-12))

    def drift(self, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        b = self.beta(t).view(-1, 1, 1, 1)
        return -0.5 * b * x

@torch.no_grad()
def sample_prob_flow_heun_eps_cfg(
    model: nn.Module,
    sde: VPSDE,
    y: torch.Tensor,                 # [B] labels 0..9
    guidance_w: float = 4.0,
    num_steps: int = 200,
    t_min: float = 1e-4,
    img_size: int = 32,
    img_channels: int = 3,
    clamp_x: bool = True,
    clamp_val: float = 2.0,
    time_power: float = 2.0,
    device: str = "cuda",
):
    """
    Probability-flow ODE with Heun integration using eps-prediction + CFG.

    score(x,t) = -eps(x,t) / sigma(t)
    dx/dt = f(x,t) - 0.5 * g(t)^2 * score(x,t)
         = drift(x,t) - 0.5*beta(t) * (-eps/sigma)
    """
    model.eval()
    B = y.shape[0]
    y = y.to(device).long()

    # time grid: more steps near small t
    u = torch.linspace(0.0, 1.0, num_steps, device=device)
    t_grid = t_min + (1.0 - t_min) * (u ** time_power)
    t_grid = torch.flip(t_grid, dims=[0])  # 1 -> t_min

    x = torch.randn(B, img_channels, img_size, img_size, device=device)

    def ode_drift(x: torch.Tensor, t_scalar: float) -> torch.Tensor:
        t = torch.full((B,), float(t_scalar), device=device, dtype=x.dtype)

        beta  = sde.beta(t).view(B, 1, 1, 1)
        sigma = sde.sigma(t).view(B, 1, 1, 1).clamp_min(1e-12)

        # unconditional (null label) via y=None
        eps_u = model(x, t, y=None)
        eps_c = model(x, t, y=y)

        eps = eps_u + guidance_w * (eps_c - eps_u)

        drift = sde.drift(x, t) - 0.5 * beta * (-eps / sigma)
        return drift

    for i in range(len(t_grid) - 1):
        t0 = t_grid[i].item()
        t1 = t_grid[i + 1].item()
        dt = t1 - t0  # negative

        k1 = ode_drift(x, t0)
        x_pred = x + dt * k1
        k2 = ode_drift(x_pred, t1)

        x = x + 0.5 * dt * (k1 + k2)

        if clamp_x:
            x = x.clamp(-clamp_val, clamp_val)

    return x

@torch.no_grad()
def sample_one_per_class(model, sde, guidance_w=4.0, num_steps=200, t_min=1e-4, device="cuda"):
    labels = torch.arange(10, device=device)
    x = sample_prob_flow_heun_eps_cfg(
        model=model,
        sde=sde,
        y=labels,
        guidance_w=guidance_w,
        num_steps=num_steps,
        t_min=t_min,
        device=device,
    )
    return x, labels

CIFAR10_NAMES = [
    "airplane", "automobile", "bird", "cat", "deer",
    "dog", "frog", "horse", "ship", "truck"
]

def to_01(x):
    """Map from [-1,1] → [0,1]."""
    return (x.clamp(-1, 1) + 1) * 0.5

def show_class_row(samples, labels=None, title="One sample per class"):
    """
    samples: [B,3,32,32] in [-1,1]
    labels:  [B] optional
    """
    x = to_01(samples.detach().cpu())
    B = x.size(0)

    plt.figure(figsize=(2 * B, 2.5))
    for i in range(B):
        plt.subplot(1, B, i + 1)
        img = x[i].permute(1, 2, 0).numpy()
        plt.imshow(img)
        plt.axis("off")
        if labels is not None:
            lbl = int(labels[i])
            plt.title(CIFAR10_NAMES[lbl], fontsize=9)

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


def show_grid(samples, nrow=4, title="Samples"):
    x = to_01(samples.detach().cpu())
    B = x.size(0)
    ncol = math.ceil(B / nrow)

    plt.figure(figsize=(2 * ncol, 2 * nrow))
    for i in range(B):
        plt.subplot(nrow, ncol, i + 1)
        img = x[i].permute(1, 2, 0).numpy()
        plt.imshow(img)
        plt.axis("off")

    plt.suptitle(title)
    plt.tight_layout()
    plt.show()


device = "cuda" if torch.cuda.is_available() else "cpu"

# --- match training hyperparams ---
num_classes = 10
time_dim = 32
base_channels = 128
img_channels = 3
attn_heads = 4
beta_min = 0.1
beta_max = 20.0
T = 1.0

ema_weights_path = "ema_weights_final_model.pth"

# Build model + load EMA weights
ema_model = UNetCIFAR3Level_Attn_CFG(
    time_dim=time_dim,
    base_channels=base_channels,
    img_channels=img_channels,
    num_classes=num_classes,
    attn_heads=attn_heads,
).to(device)

ema_sd = torch.load(ema_weights_path, map_location=device)
ema_model.load_state_dict(ema_sd, strict=True)
ema_model.eval()

# Build SDE
sde = VPSDE(beta_min=beta_min, beta_max=beta_max, T=T)




In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn

# ---------------------------
# Utilities
# ---------------------------
def _sync_if_cuda(device):
    if isinstance(device, str):
        is_cuda = "cuda" in device
    else:
        is_cuda = (device.type == "cuda")
    if is_cuda and torch.cuda.is_available():
        torch.cuda.synchronize()

def _percentiles(xs, ps=(50, 95)):
    xs = np.asarray(xs, dtype=np.float64)
    return {f"p{p}": float(np.percentile(xs, p)) for p in ps}

class TimedModel(nn.Module):
    """
    Wraps your model to:
      - count NFEs (number of model forward calls)
      - accumulate forward time using CUDA events
    """
    def __init__(self, model: nn.Module, device="cuda"):
        super().__init__()
        self.model = model
        self.device = device
        self.reset_stats()

    def reset_stats(self):
        self.nfe = 0
        self.model_ms = 0.0

    @torch.no_grad()
    def forward(self, *args, **kwargs):
        self.nfe += 1

        if torch.cuda.is_available() and ("cuda" in str(self.device)):
            start = torch.cuda.Event(enable_timing=True)
            end = torch.cuda.Event(enable_timing=True)
            start.record()
            out = self.model(*args, **kwargs)
            end.record()
            torch.cuda.synchronize()
            self.model_ms += start.elapsed_time(end)
            return out
        else:
            t0 = time.perf_counter()
            out = self.model(*args, **kwargs)
            t1 = time.perf_counter()
            self.model_ms += (t1 - t0) * 1000.0
            return out


# ---------------------------------------------------------------------
# Timed sampler wrapper for THIS notebook:
# uses your existing: sample_prob_flow_heun_eps_cfg(model, sde, y, ...)
# ---------------------------------------------------------------------
@torch.no_grad()
def sampler_prob_flow_heun_cfg_timed(
    timed_model,
    sde,
    *,
    batch_size: int,
    num_steps: int,
    guidance_w: float,
    t_min: float,
    device: str,
    time_power: float = 2.0,
    num_classes: int = 10,
):
    # Random labels 0..9 (same approach you used previously)
    y = torch.randint(low=0, high=num_classes, size=(batch_size,), device=device)

    # Call YOUR notebook's sampler function directly (important!)
    x = sample_prob_flow_heun_eps_cfg(
        model=timed_model,
        sde=sde,
        y=y,
        guidance_w=guidance_w,
        num_steps=num_steps,
        t_min=t_min,
        time_power=time_power,
        device=device,
        clamp_x=True,
    )
    return x


# ---------------------------
# Benchmark runner
# ---------------------------
def run_sampling_benchmark(
    model: nn.Module,
    sde,
    sampler_fn,
    *,
    num_steps: int,
    batch_size: int,
    device="cuda",
    t_min=1e-4,
    warmup_runs=2,
    timed_runs=10,
    reset_cuda_peak_mem=True,
    **sampler_kwargs,
):
    timed_model = TimedModel(model, device=device).to(device)

    # Warmup
    for _ in range(warmup_runs):
        timed_model.reset_stats()
        _sync_if_cuda(device)
        _ = sampler_fn(
            timed_model, sde,
            batch_size=batch_size,
            num_steps=num_steps,
            device=device,
            t_min=t_min,
            **sampler_kwargs
        )
        _sync_if_cuda(device)

    per_img_total_ms = []
    per_img_model_ms = []
    per_img_overhead_ms = []

    nfes_per_run = []
    ms_per_forward_total = []
    ms_per_forward_model = []
    throughput_img_per_s = []

    peak_alloc_mb = []
    peak_reserved_mb = []

    for _ in range(timed_runs):
        timed_model.reset_stats()

        if reset_cuda_peak_mem and torch.cuda.is_available() and ("cuda" in str(device)):
            torch.cuda.reset_peak_memory_stats()

        _sync_if_cuda(device)
        t0 = time.perf_counter()
        _ = sampler_fn(
            timed_model, sde,
            batch_size=batch_size,
            num_steps=num_steps,
            device=device,
            t_min=t_min,
            **sampler_kwargs
        )
        _sync_if_cuda(device)
        t1 = time.perf_counter()

        total_ms_batch = (t1 - t0) * 1000.0
        model_ms_batch = float(timed_model.model_ms)
        overhead_ms_batch = total_ms_batch - model_ms_batch

        nfe_batch = int(timed_model.nfe)
        nfes_per_run.append(nfe_batch)

        total_ms_img = total_ms_batch / batch_size
        model_ms_img = model_ms_batch / batch_size
        overhead_ms_img = overhead_ms_batch / batch_size

        per_img_total_ms.append(total_ms_img)
        per_img_model_ms.append(model_ms_img)
        per_img_overhead_ms.append(overhead_ms_img)

        denom = max(nfe_batch, 1)
        ms_per_forward_total.append(total_ms_batch / denom)
        ms_per_forward_model.append(model_ms_batch / denom)

        throughput_img_per_s.append(1000.0 / max(total_ms_img, 1e-12))

        if torch.cuda.is_available() and ("cuda" in str(device)):
            peak_alloc_mb.append(torch.cuda.max_memory_allocated() / (1024**2))
            peak_reserved_mb.append(torch.cuda.max_memory_reserved() / (1024**2))

    # NFE/sample: for diffusion, each sample gets all evaluations; NFE/sample == nfe_batch
    nfe_per_sample = nfes_per_run
    nfe_per_img_legacy = [n / batch_size for n in nfes_per_run]

    summary = {
        "num_steps": num_steps,
        "batch_size": batch_size,
        "t_min": t_min,
        "timed_runs": timed_runs,

        "nfe_per_sample_percentiles": _percentiles(nfe_per_sample, ps=(50, 95)),
        "nfe_per_img_legacy_percentiles": _percentiles(nfe_per_img_legacy, ps=(50,)),

        "total_ms_percentiles": _percentiles(per_img_total_ms, ps=(50, 95)),
        "model_ms_percentiles": _percentiles(per_img_model_ms, ps=(50, 95)),
        "overhead_ms_percentiles": _percentiles(per_img_overhead_ms, ps=(50, 95)),

        "ms_per_forward_total_percentiles": _percentiles(ms_per_forward_total, ps=(50, 95)),
        "ms_per_forward_model_percentiles": _percentiles(ms_per_forward_model, ps=(50, 95)),

        "throughput_img_per_s_percentiles": _percentiles(throughput_img_per_s, ps=(50, 95)),
    }

    if peak_alloc_mb:
        summary["peak_alloc_mb_p50"] = _percentiles(peak_alloc_mb, ps=(50,))["p50"]
        summary["peak_reserved_mb_p50"] = _percentiles(peak_reserved_mb, ps=(50,))["p50"]

    return summary


def pretty_print_summary(summary: dict, name=""):
    print(f"\n=== Benchmark: {name} ===")
    print(f"steps={summary['num_steps']} | batch={summary['batch_size']} | timed_runs={summary['timed_runs']}")
    print(f"NFE/sample p50: {summary['nfe_per_sample_percentiles']['p50']:.1f} | p95: {summary['nfe_per_sample_percentiles']['p95']:.1f}")
    print(f"NFE/img (legacy: NFE/sample ÷ batch) p50: {summary['nfe_per_img_legacy_percentiles']['p50']:.3f}")

    print(f"total ms/img p50: {summary['total_ms_percentiles']['p50']:.3f} | p95: {summary['total_ms_percentiles']['p95']:.3f}")
    print(f"model ms/img p50: {summary['model_ms_percentiles']['p50']:.3f} | p95: {summary['model_ms_percentiles']['p95']:.3f}")
    print(f"overhd ms/img p50: {summary['overhead_ms_percentiles']['p50']:.3f} | p95: {summary['overhead_ms_percentiles']['p95']:.3f}")

    print(
        f"ms/forward TOTAL p50: {summary['ms_per_forward_total_percentiles']['p50']:.6f} | "
        f"p95: {summary['ms_per_forward_total_percentiles']['p95']:.6f}"
    )
    print(
        f"ms/forward MODEL p50: {summary['ms_per_forward_model_percentiles']['p50']:.6f} | "
        f"p95: {summary['ms_per_forward_model_percentiles']['p95']:.6f}"
    )

    print(
        f"throughput img/s p50: {summary['throughput_img_per_s_percentiles']['p50']:.2f} | "
        f"p95: {summary['throughput_img_per_s_percentiles']['p95']:.2f}"
    )

    if "peak_alloc_mb_p50" in summary:
        print(f"peak alloc MB p50: {summary['peak_alloc_mb_p50']:.1f} | peak reserved MB p50: {summary['peak_reserved_mb_p50']:.1f}")


# ---------------------------
# Run both batch sizes (16 + 128)
# ---------------------------
GUIDANCE_W = 4.0
NUM_STEPS = 500     # as you requested
T_MIN = 1e-4
TIME_POWER = 2.0    # keep same as notebook unless you intentionally change it

bench_final_b16 = run_sampling_benchmark(
    ema_model, sde, sampler_prob_flow_heun_cfg_timed,
    num_steps=NUM_STEPS,
    batch_size=16,
    device=device,
    t_min=T_MIN,
    guidance_w=GUIDANCE_W,
    time_power=TIME_POWER,
    num_classes=10,
)
pretty_print_summary(bench_final_b16, name=f"FINAL model | Heun PF-ODE + CFG | steps={NUM_STEPS} | w={GUIDANCE_W} | batch=16")

bench_final_b128 = run_sampling_benchmark(
    ema_model, sde, sampler_prob_flow_heun_cfg_timed,
    num_steps=NUM_STEPS,
    batch_size=128,
    device=device,
    t_min=T_MIN,
    guidance_w=GUIDANCE_W,
    time_power=TIME_POWER,
    num_classes=10,
)
pretty_print_summary(bench_final_b128, name=f"FINAL model | Heun PF-ODE + CFG | steps={NUM_STEPS} | w={GUIDANCE_W} | batch=128")


NameError: name 'ema_model' is not defined